<a href="https://colab.research.google.com/github/OliBomby/Mapperatorinator/blob/main/colab/mapperatorinator_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beatmap Generation with Mapperatorinator

This notebook is an interactive demo of an osu! beatmap generation model created by OliBomby. This model is capable of generating hit objects, hitsounds, timing, kiai times, and SVs for all 4 gamemodes. You can upload a beatmap to give to the model as additional context or remap parts of the beatmap.

### Instructions for running:

* Read and accept the rules regarding using this tool
* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Press ▶️ on the left of each cell to execute the cell
* __Upload Audio__, choose a .mp3 or .ogg file from your computer
* __Upload Beatmap__, optionally choose a .osu file from your computer
* __Configure__, choose your generation parameters to control the style of the generated beatmap
* Generate the beatmap using the __Generate Beatmap__ cell (it may take a few minutes depending on the length of the song)


In [ ]:
#@title Setup Environment { display-mode: "form" }
#@markdown ### Use this tool responsibly. Always disclose the use of AI in your beatmaps. Do not upload the generated beatmaps.
i_accept_the_rules = False # @param {type:"boolean"}

assert i_accept_the_rules, "Read and accept the rules first!"

!git clone https://github.com/OliBomby/Mapperatorinator.git
%cd Mapperatorinator

!pip install -r requirements.txt

from google.colab import files

import os
import glob
from hydra import compose, initialize
from osuT5.osuT5.event import ContextType
from inference import main

output_path = "output"
input_audio = ""
input_beatmap = ""

In [ ]:
#@title Upload Audio { display-mode: "form" }

def upload_audio():
  data = list(files.upload().keys())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return data[0]

input_audio = upload_audio()

In [ ]:
#@title (Optional) Upload Beatmap { display-mode: "form" }
use_reference_beatmap = False # @param {type:"boolean"}

def upload_beatmap():
  data = list(files.upload().keys())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return data[0]

if use_reference_beatmap:
    input_beatmap = upload_beatmap()
else:
    input_beatmap = ""

In [ ]:
#@title Configure and Generate Beatmap { display-mode: "form" }

# @markdown #### You can input -1 to leave the value unknown.
# @markdown ---
gamemode = "standard" # @param ["standard", "taiko", "catch the beat", "mania"]
difficulty = 5 # @param {type:"number"}
mapper_id = -1 # @param {type:"integer"}
year = 2023 # @param {type:"integer"}
hitsounded = True # @param {type:"boolean"}
slider_multiplier = 1.4 # @param {type:"slider", min:0.4, max:3.6, step:0.1}
circle_size = 4 # @param {type:"number"}
keycount = 4 # @param {type:"slider", min:1, max:18, step:1}
hold_note_ratio = -1 # @param {type:"number"}
scroll_speed_ratio = -1 # @param {type:"number"}
descriptor_1 = '' # @param ["2B", "slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "aspire", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
descriptor_2 = '' # @param ["2B", "slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "aspire", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
descriptor_3 = '' # @param ["2B", "slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "aspire", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
negative_descriptor_1 = '' # @param ["2B", "slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "aspire", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
negative_descriptor_2 = '' # @param ["2B", "slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "aspire", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
negative_descriptor_3 = '' # @param ["2B", "slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "aspire", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
# @markdown ---
add_to_beatmap = False # @param {type:"boolean"}
start_time = -1 # @param {type:"integer"}
end_time = -1 # @param {type:"integer"}
in_context = "[NONE]" # @param ["[NONE]", "[TIMING]", "[TIMING,KIAI]", "[TIMING,KIAI,MAP]", "[GD,TIMING,KIAI]", "[NO_HS,TIMING,KIAI]"]
output_type = "[TIMING,KIAI,MAP,SV]" # @param ["[TIMING,KIAI,MAP,SV]", "[TIMING]"]
cfg_scale = 1 # @param {type:"slider", min:1, max:10, step:0.1}
super_timing = True # @param {type:"boolean"}
seed = 0 # @param {type:"integer"}
# @markdown ---

# Get actual parameters
a_gamemode = ["standard", "taiko", "catch the beat", "mania"].index(gamemode)
a_difficulty = None if difficulty == -1 else difficulty
a_mapper_id = None if mapper_id == -1 else mapper_id
a_year = None if year == -1 else year
a_circle_size = None if circle_size == -1 else circle_size
a_hold_note_ratio = None if hold_note_ratio == -1 else hold_note_ratio
a_scroll_speed_ratio = None if scroll_speed_ratio == -1 else scroll_speed_ratio
descriptors = [d for d in [descriptor_1, descriptor_2, descriptor_3] if d != '']
negative_descriptors = [d for d in [negative_descriptor_1, negative_descriptor_2, negative_descriptor_3] if d != '']

a_start_time = None if start_time == -1 else start_time
a_end_time = None if end_time == -1 else end_time
a_in_context = [ContextType(c.lower()) for c in in_context[1:-1].split(',')]
a_output_type = [ContextType(c.lower()) for c in output_type[1:-1].split(',')]

# Validate stuff
if any(c in a_in_context for c in [ContextType.TIMING, ContextType.KIAI, ContextType.MAP, ContextType.SV, ContextType.GD, ContextType.NO_HS]) or add_to_beatmap:
    assert os.path.exists(input_beatmap), "Please upload a reference beatmap."
assert os.path.exists(input_audio), "Please upload an audio file."
    
# Create config
with initialize(version_base="1.1", config_path="configs"):
    conf = compose(config_name="inference_v29")

# Do inference
conf.audio_path = input_audio
conf.output_path = output_path
conf.beatmap_path = input_beatmap
conf.gamemode = a_gamemode
conf.difficulty = a_difficulty
conf.mapper_id = a_mapper_id
conf.year = a_year
conf.hitsounded = hitsounded
conf.slider_multiplier = slider_multiplier
conf.circle_size = a_circle_size
conf.keycount = keycount
conf.hold_note_ratio = a_hold_note_ratio
conf.scroll_speed_ratio = a_scroll_speed_ratio
conf.descriptors = descriptors
conf.negative_descriptors = negative_descriptors
conf.add_to_beatmap = add_to_beatmap
conf.start_time = a_start_time
conf.end_time = a_end_time
conf.in_context = a_in_context
conf.output_type = a_output_type
conf.cfg_scale = cfg_scale
conf.super_timing = super_timing
conf.seed = seed

main(conf)

# Download file
def get_most_recent_file(folder_path):
    # Get a list of all files in the folder
    files = glob.glob(os.path.join(folder_path, "*"))
    
    # Ensure there are files in the folder
    if not files:
        return None  # No files in the folder

    # Find the most recently created file
    most_recent_file = max(files, key=os.path.getctime)
    return most_recent_file

if conf.add_to_beatmap:
    files.download(conf.beatmap_path)
else:
    files.download(get_most_recent_file(output_path))
